# Pytorch-lightning + LSTM

Following is an example how one can rewrite pytorch LSTM model in a Lightning form. The goal is to create 'trainer' instance from Trainer class of pytorch lightning. A ```Trainer``` consists of ```System``` part and ```DataModule``` part. ```System``` part consists of ```Model``` part (including forward method) and training part (including loss function, optimizer etc.). ```DataModule``` consists of ```Dataloader``` for training, validation and test dataset. The datasets are defined by pytorch ```Dataset``` class. In short, the relations between each class are 


`
Trainer --+-- System -----+- Model (network definition and forward method)
          |               | 
          |               +- training scheme (optimizer, loss function etc.)
          | 
          +-- Data Module -- Dataloader
                             +-- DataSet       
`


In the example below, the components are named as follows. 

```
IndoorDataset3    : Dataset
IndoorLSTM       : Model 
IndoorDataModule : DataModule
IndoorSystem     : System to feed to trainer (includes training system inside)
```

## Sumamry

- Lightning system can be defined by Model + training scheme separately (in most examples available in the internet, ```Model``` (=network) definition and the training scheme are put in one big ```System``` class). 

- I tried to perform inference, either using ```trainer.test()``` or ```trainer.predict()```. Somehow neither of them works, and I ended up using a callback to store the best model during the training, and reload it specifically, for the inference. There must be a smarter way. 


## Acknowledgement

I used the pytorch LSTM model created by @captainqxy. <br>
**LSTM by pytorch with Unified Wi-Fi Feats** <br>
https://www.kaggle.com/luffy521/lstm-by-pytorch-with-unified-wi-fi-feats

The model above is based on <br>
**LSTM by Keras with Unified Wi-Fi Feats** by @kokitanisaka <br>
https://www.kaggle.com/kokitanisaka/lstm-by-keras-with-unified-wi-fi-feats

I used the data published by @kokitanisaka <br>
**Make dataset with Wi-Fi and Beacon** <br>
https://www.kaggle.com/kokitanisaka/make-dataset-with-wi-fi-and-beacon

Thank you so much, @captainqxy and  @kokitanisaka.

Here the packages are imported.

In [ ]:
import os
from pathlib import Path
import warnings
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.datamodule import LightningDataModule
from pytorch_lightning.core.lightning import LightningModule

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datatable import (dt, f, join)
import gc

import numpy as np
import pandas as pd
import logging

import random

Make logging a bit quiet. Set device. Do some housekeeping stuff. 

In [ ]:
#---------------------------------------------------------
#logging.getLogger('pytorch_lightning').setLevel(logging.CRITICAL)
logging.getLogger('lightning').setLevel(logging.CRITICAL)
warnings.filterwarnings('ignore')
# ========================================================
gc.enable()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999
dt.options.display.max_nrows = 999

# =========================================================
SEED = 2021

Here come the dataset and the data loader. They are combined in ```DataModule``` of Lightning. 

In [ ]:

# =========================================================
# data set / data module / data loader / model / system
# =========================================================


class IndoorDataset3(Dataset):
    def __init__(self, data, N_FEAT, flag='TRAIN'):
        self.data = data
        self.n_feat = N_FEAT
        self.flag = flag

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):

        db = self.data[index, : self.n_feat].astype(np.long)
        dr = self.data[index, self.n_feat: (
            self.n_feat * 2)].astype(np.float32)
        d_xy = self.data[index, (self.n_feat * 2): -1].astype(np.float32)
        d_floor = self.data[index, -1].astype(np.float32)

        if self.flag == 'TRAIN':
            return db, dr, d_xy, d_floor
        else:
            return db, dr

# ============================================

class IndoorDataModule(LightningDataModule):

    def __init__(self, data, test_data, tix, vix, bx, rx, N_FEAT, BS):
        super().__init__()

        self.data = data
        self.test_data = test_data
        self.tix = tix
        self.vix = vix

        self.bx = bx
        self.rx = rx

        self.N_FEAT = N_FEAT
        self.BS = BS

        if Path('.').cwd() == Path('/home/meg/k5'):
            self.num_cores = multiprocessing.cpu_count()
        else:
            self.num_cores = 0

    def prepare_data(self):

        self.data_npy = self.data[:, self.bx +
                                  self.rx+['x', 'y', 'floor']].to_numpy()
        self.test_data_npy = self.test_data[:,
                                            self.bx+self.rx+['x', 'y', 'floor']].to_numpy()

    def train_dataloader(self):

        train_ds = IndoorDataset3(
            self.data_npy[self.tix, :], self.N_FEAT, 'TRAIN')
        train_dl = DataLoader(train_ds, batch_size=self.BS,
                              shuffle=True, drop_last=False, num_workers=self.num_cores)

        return train_dl

    def val_dataloader(self):
        valid_ds = IndoorDataset3(
            self.data_npy[self.vix, :], self.N_FEAT, 'TRAIN')
        valid_dl = DataLoader(valid_ds, batch_size=self.BS,
                              shuffle=False, drop_last=False, num_workers=self.num_cores)
        return valid_dl

    def test_dataloader(self):
        test_ds = IndoorDataset3(self.test_data_npy, self.N_FEAT, 'TEST')
        test_dl = DataLoader(test_ds, batch_size=self.BS,
                             shuffle=False, num_workers=self.num_cores)
        return test_dl



A LSTM model was copied from **LSTM by pytorch with Unified Wi-Fi Feats** <br>
https://www.kaggle.com/luffy521/lstm-by-pytorch-with-unified-wi-fi-feats

The model is edited from the original to the one with 4 times smaller layers.

In [ ]:
# ============================================

class IndoorLSTM(LightningModule):

    def __init__(self, embedding_dim=64, wifi_bssids_size=16, N_FEAT=4):

        super().__init__()

        self.N_FEAT = N_FEAT

        self.emb_BSS = nn.Embedding(wifi_bssids_size, embedding_dim)

        self.lstm1 = nn.LSTM(input_size=256, hidden_size=128,
                             dropout=0.3, bidirectional=False)

        self.lstm2 = nn.LSTM(input_size=128, hidden_size=16,
                             dropout=0.1, bidirectional=False)
        self.lr = nn.Linear(self.N_FEAT, self.N_FEAT * embedding_dim)
        self.lr1 = nn.Linear(self.N_FEAT * embedding_dim * 2, 256)
        self.lr_xy = nn.Linear(16, 2)

        self.lr_floor = nn.Linear(16, 1)
        self.batch_norm1 = nn.BatchNorm1d(self.N_FEAT)
        self.batch_norm2 = nn.BatchNorm1d(self.N_FEAT * embedding_dim * 2)

        self.batch_norm3 = nn.BatchNorm1d(1)
        self.dropout = nn.Dropout(0.3)


    def forward(self, xb, xr):

        x_bssid = self.emb_BSS(xb)

        x_bssid = torch.flatten(x_bssid, start_dim=-2)

        x_rssi = self.batch_norm1(xr)
        x_rssi = self.lr(x_rssi)
        x_rssi = torch.relu(x_rssi)

        x = torch.cat([x_bssid, x_rssi], dim=-1)

        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = self.lr1(x)
        x = torch.relu(x)

        x = x.unsqueeze(-2)
        x = self.batch_norm3(x)
        x = x.transpose(0, 1)

        x, _ = self.lstm1(x)
        x = x.transpose(0, 1)
        x = torch.relu(x)
        x = x.transpose(0, 1)

        x, _ = self.lstm2(x)
        x = x.transpose(0, 1)
        x = torch.relu(x)

        xy = self.lr_xy(x)
        floor = self.lr_floor(x)
        floor = torch.relu(floor)

#        return xy.squeeze(-2), floor.squeeze(-2)
        return xy.squeeze(-2)




The ```System``` class to feed to a Lightning trainer. The learning rate is reduced from ```LR``` to ```LR2``` by multiplying a single constant factor, ```gamma```, at each epoch. 

In [ ]:
# ========================================================================


class IndoorSystem(LightningModule):

    def __init__(self, model=None, fold=0, LR=0.1, LR2=1e-4, EP=16, LOG=None):
        super().__init__()

        self.model = model
        self.fold = fold
        self.best_score = 1000
        self.best_loss = 1000 * 1000
        self.best_epoch = -1

        self.learning_rate = LR
        self.LR1 = LR
        self.LR2 = LR2
        self.EP = EP

        self.ff = LOG
    # --------------------------------------------

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.learning_rate, weight_decay=1e-2)

        gamma = (self.LR2/self.LR1) ** (1.0 / self.EP)

        scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=gamma),
                     'interval': 'epoch'}

        return [optimizer], [scheduler]

    # --------------------------------------------
    def training_step(self, batch, batch_idx):

        db, dr, d_xy, _ = batch
        xy = self.model(db, dr)

        mse = nn.MSELoss()
        loss = mse(xy, d_xy)

        print(f"\r\033[32mfold \033[0m{self.fold} ", end='')
        print(
            f"\033[33mEPOCH\033[0m{self.current_epoch: 5} ", end='')
        print(f"\033[31midx\033[0m{batch_idx: 3} ", end='')
        print(f"\033[34mloss\033[0m{loss: 10.3f} ", end='')

        with open(self.ff, 'a') as ff:

            print(f"fold{self.fold} ", end='', file=ff)
            print(f"EPOCH{self.current_epoch:5} ", end='', file=ff)
            print(f'idx{batch_idx:3} ', end='', file=ff)
            print(f'loss{loss: 10.3f} ', file=ff)

        return {'loss': loss}

    # --------------------------------------------
    def validation_step(self, batch, batch_idx):
        db, dr, d_xy, d_floor = batch
        xy = self.model(db, dr)

        mse = nn.MSELoss()
        val_loss = mse(xy, d_xy)

        return {'loss': val_loss}

    # --------------------------------------------
    def training_epoch_end(self, outputs):
        opt = self.optimizers()
        self.learning_rate = opt.param_groups[0]['lr']

    # --------------------------------------------
    def validation_epoch_end(self, outputs):

        loss = torch.stack([f['loss'] for f in outputs]).mean()

        if self.best_loss > loss:
            #            self.best_score = score
            self.best_loss = loss
            self.best_epoch = self.current_epoch

        if self.current_epoch != 0:
            pass

#            print(f"\033[34mv\033[0m{self.best_loss:10.3f} ", end='')
#            print(f"\033[32mepoch\033[0m{self.best_epoch:5} ", end='')
#            print(f"\033[35mlr\033[0m{self.learning_rate:7.4f} ", end='')
#            print("")

#            with open(self.ff, 'a') as ff:
#                print(
#                    f'\033[34mv\033[0m{self.best_loss:10.3f} ', end='', file=ff)
#                print(
#                    f"\033[32mepoch\033[0m{self.best_epoch:5} ", end='', file=ff)
#                print(
#                    f"\033[35mlr\033[0m{self.learning_rate:7.4f} ", end='', file=ff)
#                print("", file=ff)
         
        self.log('val_loss', loss)


# ========================================================================
# class completed |  end of data module / data loader / model / system
# ========================================================================



Utilities. Note that the error in the floor predictions are not included in the metric.

In [ ]:

# ============================================
#  utilities functions
# ============================================

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def comp_metric2(xhat, yhat, fhat, x, y, f):
#    intermediate = np.sqrt((xhat-x)**2 + (yhat-y)**2) + 15 * np.abs(fhat-f)

    intermediate = np.sqrt((xhat-x)**2 + (yhat-y)**2)
    return intermediate.sum()/xhat.shape[0]


Convert the raw site-data in the form that one can feed to ```Dataset```. The training data consists of two parts: 

> columns of ```bx``` : the IDs of WiFi access points. They are converted to integers. <br>
columns of ```rx``` : the strengths of the signals that the smart phone received from each WiFi access point. Scaled to a normal (=Gaussian) distribution (average=0, standard deviation = 1). <br>

Several additional informations are returned from ```wifi_prep3``` in the same time. 


> ```wifi_bssids```: list of the names of the all WiFi access points installed in the building (=site). <br>
```le```: a level encoder to convert string names of ```wifi_bssids``` to integers. <br>
```ss```: a scaler for the WiFi signals to those in a normal distribution. The average value is subtracted from all measurements, and then divided by the standard deviation. <br>

```data``` and ```test_data``` are for training (+validation) dataset and test (=inference) dataset, respectively.  Both are in the form of datatable. 


In [ ]:
# ========================================================
def wifi_prep3(data_site, test_data_site, N_FEAT):

    bx = [i for i in data_site.names if i.startswith('wifi_bssid_')]
    rx = [i for i in data_site.names if i.startswith('wifi_rssi_')]

    dtmp = data_site[:, bx].copy()
    dtmp.rbind(test_data_site[:, bx])

    wifi_bssids = np.unique(dtmp.to_numpy())
    wifi_bssids_size = len(wifi_bssids)

    del dtmp
    gc.collect()

    timegapx = [i for i in data_site.names if i.startswith('wifi_timegap_')]
    beaconx = [i for i in data_site.names if i.startswith('beacon_')]
#    label_cols = ['site', 'path', 'timestamp', 'x', 'y', 'floor']
    label_cols = ['site', 'path', 'x', 'y', 'floor']


    # level encoder
    le = LabelEncoder()
    _ = le.fit(wifi_bssids)

    ss = StandardScaler()
    _ = ss.fit(data_site[:, rx])

    data = data_site.copy()
    data[:, rx] = ss.transform(data_site[:, rx])

    for i in bx:
        data[:, i] = le.transform(data_site[:, i])

    test_data = test_data_site.copy()
    test_data[:, rx] = ss.transform(test_data_site[:, rx])

    for i in bx:
        test_data[:, i] = le.transform(test_data_site[:, i])
 
    # reshape
    data = data[:, label_cols + bx + rx + timegapx + beaconx]
    data = data[:, f[:].remove(f[bx[N_FEAT]:bx[-1]])]
    data = data[:, f[:].remove(f[rx[N_FEAT]:rx[-1]])]
    data = data[:, f[:].remove(f[timegapx[0]:timegapx[-1]])]
    data = data[:, f[:].remove(f[beaconx[0]:beaconx[-1]])]

#    test_data = test_data[:, label_test_cols + bx + rx + timegapx + beaconx]
    test_data = test_data[:, label_cols + bx + rx + timegapx + beaconx]
    test_data = test_data[:, f[:].remove(f[bx[N_FEAT]:bx[-1]])]
    test_data = test_data[:, f[:].remove(f[rx[N_FEAT]:rx[-1]])]
    test_data = test_data[:, f[:].remove(f[timegapx[0]:timegapx[-1]])]
    test_data = test_data[:, f[:].remove(f[beaconx[0]:beaconx[-1]])]

    bx = bx[: N_FEAT]
    rx = rx[: N_FEAT]

    return data, test_data, wifi_bssids, wifi_bssids_size, le, ss, bx, rx



The main loop. For each site (=building), models are calculated using 5-fold-split training dataset. The hyperparameters (number of epochs and so on) are not optimized.

In [ ]:
# ============================================
#  directories

path = Path('../input/unified-ds-wifi-and-beacon/')
log_path = Path('.')
TRAIN = path
TEST = path
LOG = log_path/'log_v1.txt'
MODEL = log_path/'model_v1'
MODEL.mkdir(exist_ok=True)

# ============================================
train_list = list(sorted(TRAIN.glob('5*.csv')))
test_data_all = dt.fread(TEST/'test.csv')

set_seed(SEED)
# ========================================================
N_SPLITS = 5

config = dict(
    N_FEAT=40,
    BS=1024,
    EP=8,
    LR=0.1, LR2=1e-3)

# ========================================================
i_sx = [0, 1]  # try first 2 sites.

# ========================================================
#  Main Loop
# ========================================================
for i_site, train_file in [(i, train_list[i]) for i in i_sx]:

    site = train_file.stem.split('_')[0]

    print(f"\033[35msite \033[31m {i_site:2} \033[0m{site}")

    with open(LOG, 'a') as ff:
        print(f"site {i_site:2} {site}", file=ff)

    # --------------------------------------
    N_FEAT = config['N_FEAT']
    BS = config['BS']
    EP = config['EP']
    LR = config['LR']
    LR2 = config['LR2']
    # --------------------------------------

    data_site = dt.fread(train_file)
    test_data_site = test_data_all[f.site == site, :]

    data, test_data, wifi_bssids, wifi_bssids_size, le, ss, bx, rx = wifi_prep3(
        data_site, test_data_site, N_FEAT)


    gkf = GroupKFold(N_SPLITS)
    for fold, (tix, vix) in enumerate(gkf.split(data, data[:, ['x', 'y', 'floor']],
                                                groups=data[:, 'path'])):

        dm = IndoorDataModule(data, test_data, tix, vix, bx, rx, N_FEAT, BS)

        model = IndoorLSTM(
            embedding_dim=8, wifi_bssids_size=wifi_bssids_size, N_FEAT=N_FEAT)

        indoor_system = IndoorSystem(
            model=model, fold=fold, LR=LR, LR2=LR2, EP=EP, LOG=LOG)

    # ======================

        checkpoint_callback = ModelCheckpoint(
            #            monitor='val_score',
            monitor='val_loss',
            dirpath=MODEL,
            filename='m-{epoch:02d}-{val_loss:.2f}',
            save_top_k=3,
            mode='min'
        )

        trainer = Trainer(
            gpus=0,
            max_epochs=EP,
            logger=False,
            callbacks=[checkpoint_callback],
            checkpoint_callback=True,
            progress_bar_refresh_rate=0)

        trainer.fit(indoor_system, dm)
        print('')

    # ======================
    #   inference

        trained_model = IndoorSystem.load_from_checkpoint(
            checkpoint_callback.best_model_path,
            model=model, fold=fold, LR=LR, LR2=LR2, EP=EP)

        _ = trained_model.eval()

        xy_stack = []
        for db, dr in dm.test_dataloader():
            xy = trained_model.model(db, dr).cpu().detach().numpy()
            xy_stack.append(xy)

        pred_xy = np.vstack(xy_stack)

    print(pred_xy.shape)

# ========================================================

